# Data Cleaning

### Libraries

In [ ]:
import pandas as pd
import numpy as np
import re

### Data import

In [ ]:
r = pd.read_csv('ristoranti_comuni.csv')

In [ ]:
r = r.drop('Unnamed: 0', axis=1)

### Data understanding

In [ ]:
r.head()

,name,url,rating,price,address,contact
0,Bar Pasticceria Marika,https://www.tripadvisor.it/Restaurant_Review-g...,"3,5",€€-€€€,"Corso Milano, 17, 24020 Selvino Italia",+39 393 250 5982
1,Ristorante Sorriso,https://www.tripadvisor.it/Restaurant_Review-g...,"4,5",€€€€,"Via Talpino 79, 24020 Selvino Italia",+39 035 763104
2,Pizzeria Birreria Roma,https://www.tripadvisor.it/Restaurant_Review-g...,"3,5",€€-€€€,"Via Monte Alben, 13, 24020 Selvino Italia",+39 035 763012
3,Ca' di Bore,https://www.tripadvisor.it/Restaurant_Review-g...,"4,0",€€-€€€,"Via Cantul 38, 24020 Aviatico Italia",+39 035 779167
4,A Casa del Contadino,https://www.tripadvisor.it/Restaurant_Review-g...,"4,0",€,"Corso Milano, 91, 24020 Selvino Italia",+39 035 764202


In [ ]:
r.shape

(1003, 6)

In [ ]:
r.dtypes

name       object
url        object
rating     object
price      object
address    object
contact    object
dtype: object

### Cleaning

Remove duplicated rows

In [ ]:
# count duplicates
r.duplicated().sum()

104

In [ ]:
# remove duplicates
r = r.loc[~r.duplicated()]

In [ ]:
r = r.reset_index(drop=True)

Look for null values (note that the scraping code placed the string 'sconosciuto' inplace of missing values).

In [ ]:
r.eq('sconosciuto').sum()

name       0
url        0
rating     0
price      0
address    0
contact    0
dtype: int64

#### Address

Create new columns 'comune', 'via' and 'CAP' by splitting the 'address' column.

In [ ]:
r['address']=r['address'].str.replace(',','')

In [ ]:
r[['via', 'CAP', 'comune']] = r['address'].apply(lambda x: pd.Series(re.split(r'(\b\d{5}\b)', x)[0:3]))


In [ ]:
r = r.drop('address', axis=1)

In [ ]:
r.head()

,name,url,rating,price,contact,via,CAP,comune
0,Bar Pasticceria Marika,https://www.tripadvisor.it/Restaurant_Review-g...,"3,5",€€-€€€,+39 393 250 5982,Corso Milano 17,24020,Selvino Italia
1,Ristorante Sorriso,https://www.tripadvisor.it/Restaurant_Review-g...,"4,5",€€€€,+39 035 763104,Via Talpino 79,24020,Selvino Italia
2,Pizzeria Birreria Roma,https://www.tripadvisor.it/Restaurant_Review-g...,"3,5",€€-€€€,+39 035 763012,Via Monte Alben 13,24020,Selvino Italia
3,Ca' di Bore,https://www.tripadvisor.it/Restaurant_Review-g...,"4,0",€€-€€€,+39 035 779167,Via Cantul 38,24020,Aviatico Italia
4,A Casa del Contadino,https://www.tripadvisor.it/Restaurant_Review-g...,"4,0",€,+39 035 764202,Corso Milano 91,24020,Selvino Italia


In [ ]:
# delete 'Italia' from the city name
r['comune']=r['comune'].str.replace(' Italia', '')

In [ ]:
# remove the restaurants in Switzerland
r = r[~r['via'].str.contains('Svizzera')]

In [ ]:
# sottoinsieme delle righe per cui non è presente il CAP nell'indirizzo
righe_cap_nullo = r[r['CAP'].isna()]

In [ ]:
len(righe_cap_nullo['via'].unique())

66

In [ ]:
comuni_distinct = r['comune'].unique().tolist()

In [ ]:
comuni_distinct = list(filter(lambda x: not pd.isna(x), comuni_distinct))


In [ ]:
comuni_distinct = list(filter(lambda x: x != "", comuni_distinct))


In [ ]:
comuni_distinct = list(filter(lambda x: x != " ", comuni_distinct))

In [ ]:
comuni_distinct

[' Selvino',
 ' Aviatico',
 ' Bergamo',
 ' Zogno',
 ' Foresto Sparso',
 ' Costa di Serina',
 ' Algua',
 ' Nembro',
 ' Gazzaniga',
 ' Ponte di Legno',
 ' Temù',
 ' Villa Dalegno Temù',
 ' Piazzatorre',
 ' Santa Brigida',
 ' Olmo al Brembo',
 ' Mezzoldo',
 ' Branzi',
 ' Piazza Brembana',
 ' Morbegno',
 ' Pian delle Betulle Margno',
 ' Crandola Valsassina',
 ' Casargo',
 ' Livigno',
 ' Margno',
 ' Breno',
 ' Niardo',
 ' Clusone',
 ' Sellero',
 ' Pisogne',
 ' Piancogno',
 ' Moggio',
 ' Cassina Valsassina',
 ' Barzio',
 ' Piani di Artavaggio',
 " Monticelli d'Ongina",
 ' Foppolo',
 ' San Pellegrino Terme',
 ' Valleve',
 ' Isola di Fondra',
 ' Bracca',
 " Moio de' Calvi",
 ' Valnegra',
 ' Camerata Cornello',
 ' Lenna',
 ' Edolo',
 ' Passo del Tonale',
 ' Sonico',
 ' Vione',
 ' Bagolino',
 ' Ponte Caffaro Bagolino',
 ' Anfo',
 ' Trento',
 ' Bardolino',
 ' Vestone',
 ' Parona Verona',
 ' Salò',
 ' Ospitaletto',
 ' Orzinuovi',
 ' Lovere',
 ' Cologne',
 ' Chiesa In Valmalenco',
 ' San Giuseppe C

In [ ]:
for index, riga in righe_cap_nullo.iterrows():
    via = riga['via']

    # Verifica se 'via' contiene uno dei comuni
    for comune in comuni_distinct:
        if comune in via:
            # Splitta 'via' in corrispondenza del nome del comune cercando la corrispondenza da destra
            corrispondenze = re.findall(r'^(.*?)(\b{}\b.*)$'.format(re.escape(comune)), via)

            if corrispondenze:
                parte_con_comune = corrispondenze[0][1].strip()
                parte_senza_comune = corrispondenze[0][0].strip()

                # Aggiorna i valori nella colonna 'comune' e 'via' del DataFrame 'r'
                r.at[index, 'comune'] = parte_con_comune
                r.at[index, 'via'] = parte_senza_comune

In [ ]:
r.loc[r['CAP'].isna(), 'comune'].unique()

array(['Ponte di Legno Italia', 'Piazzatorre Italia', 'Margno Italia',
       'Moggio Italia', 'Cremeno Italia', 'Cassina Valsassina Italia',
       'Carona Italia', 'Valleve Italia', 'Lenna Italia', 'Branzi Italia',
       'Temù Italia', 'Bagolino Italia', 'Chiesa In Valmalenco Italia',
       'Caspoggio Italia', 'Pisogne Italia', 'Lovere Italia',
       'Lanzada Italia', 'Campodolcino Italia', nan, 'Brescia Italia',
       'Montecampione Artogne Italia', 'Artogne Italia', 'Aprica Italia',
       'Valfurva Italia', 'San Pellegrino Terme Italia', 'Barzio Italia',
       'San Giovanni Bianco Italia', 'Valdisotto Italia',
       'San Colombano Tel 3289145398 Valdisotto Italia', 'Lecco Italia',
       'Bormio Italia', 'Capriano del Colle Italia',
       'Oltre il Colle Italia', 'Madesimo Italia', 'Songavazzo Italia',
       'Corteno Golgi Italia', 'Valdidentro Italia', 'Valbondione Italia',
       'Collio Italia', 'Mapello Italia',
       'Castione della Presolana Italia'], dtype=object)

In [ ]:
for index, riga in righe_cap_nullo.iterrows():
    comune = riga['comune']

    # Cerca il CAP nelle righe con lo stesso comune
    righe_stesso_comune = r[r['comune'] == comune]
    cap_stesso_comune = righe_stesso_comune['CAP'].dropna()

    if not cap_stesso_comune.empty:
        # Copia il CAP dalla prima riga con lo stesso comune
        cap_copiato = cap_stesso_comune.iloc[0]

        # Aggiorna il valore nella colonna 'CAP' del DataFrame 'r'
        r.at[index, 'CAP'] = cap_copiato

In [ ]:
r['comune']=r['comune'].str.replace(' Italia', '')

__contact__

In [ ]:
# delete spaces from tel. numbers and replace non-numerical strings with empty string
r['contact']=r['contact'].str.replace(' ','')
r['contact'] = r['contact'].apply(lambda x: np.nan if not re.search(r'\d',x) else x)

__rating__

Per controllo diretto, risulta che i ristoranti con rating = -1,0 sono quelli con zero recensioni.

In [ ]:
r['rating'].unique()

array(['3,5', '4,5', '4,0', '3,0', '-1,0', '2,5', '5,0', '2,0', '1,5',
       '1,0'], dtype=object)

In [ ]:
r.loc[r['rating']=='-1,0']

,name,url,rating,price,contact,via,CAP,comune
23,Bar Station Selvino,https://www.tripadvisor.it/Restaurant_Review-g...,"-1,0",€,+3903519837293,Via Enea Talpino 20 Ip Benzinaio,24020,Selvino
77,Pizzeria La Barbolena,https://www.tripadvisor.it/Restaurant_Review-g...,"-1,0",Italiana,NaN,Localita Pian delle Betulle,NaN,Margno
95,I Portici Griglieria Pizzeria,https://www.tripadvisor.it/Restaurant_Review-g...,"-1,0",Pizza,+390364088059,Via Canevali 14,25043,Breno
330,Carona (val Carisole),https://www.tripadvisor.it/Restaurant_Review-g...,"-1,0",Italiana,+393288142888,Val Carisole,NaN,Carona
554,GiboGel,https://www.tripadvisor.it/Restaurant_Review-g...,"-1,0",€,+393407865285,Via Vittorio Veneto 20 Presso centro commercia...,24060,Rogno


In [ ]:
# replace -1,0 ratings with NaN
r['rating']=r['rating'].replace('-1,0', np.nan)

# replace ',' with '.' and convert str to float. empty str are set to NaN
r['rating']=r['rating'].str.replace(',','.')
r['rating'] = pd.to_numeric(r['rating'], errors='coerce')

__price__

In [ ]:
r['price'].unique()

array(['€€-€€€', '€€€€', '€', 'Pizza', 'Pasticcerie e gelaterie',
       'Italiana', 'Pub', 'Lombarda', 'Italiana (nord)', 'Bar', 'Forni'],
      dtype=object)

In [ ]:
# replace wrong istances with NaN
r['price'] = np.where(r['price'].str.contains('€'), r['price'], np.nan)

In [ ]:
r['price']

0      €€-€€€
1        €€€€
2      €€-€€€
3      €€-€€€
4           €
        ...  
894      €€€€
895       NaN
896    €€-€€€
897         €
898    €€-€€€
Name: price, Length: 884, dtype: object

Possible outcomes of 'price' variable are: € = 'cibo economico', €€-€€€ = 'fascia media',
                                           €€€€ = 'cucina raffinata'

In [ ]:
# change the values of 'price' variable with coresponding description
r['price'] = r['price'].replace({'€': 'cibo economico',
                                  '€€-€€€': 'fascia media',
                                  '€€€€': 'cucina raffinata'})

## Data Export

In [ ]:
r.to_csv('ristoranti_tripadvisor.csv', index = False)

In [ ]:
r.to_excel('ristoranti_tripadvisor.xlsx', index = False)